## Libraries Installation

### NLTK Installation

In [1]:
%pip install nltk

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     ---------------------------- --------- 30.7/41.5 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 401.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.5 MB 5.1 MB/s eta 0:00:01
   ---- ----------------------------------- 0.2/1.5 MB 3.1 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.5 MB 2.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.5 MB 2.4 MB/s eta 0:00:01
   ---------- ----------------------------- 0.4/1.5 MB 2.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.5 MB 2.3 MB/s eta 0:00:01
   -------------- 


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### ir_datasets Installation

In [3]:
%pip install ir_datasets

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     -------------------------------------- 60.9/60.9 kB 649.3 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml):


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### PyQt6 Installation

In [4]:
%pip install PyQt6

   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.7 MB 435.7 kB/s eta 0:00:59
   ---------------------------------------- 0.1/25.7 MB 787.7 kB/s eta 0:00:33
   ---------------------------------------- 0.1/25.7 MB 1.1 MB/s eta 0:00:24
   ---------------------------------------- 0.1/25.7 MB 798.9 kB/s eta 0:00:33
   ---------------------------------------- 0.2/25.7 MB 766.6 kB/s eta 0:00:34
   ---------------------------------------- 0.2/25.7 MB 1.0 MB/s eta 0:00:25
   ---------------------------------------- 0.3/25.7 MB 1.1 MB/s eta 0:00:24
    --------------------------------------- 0.3/25.7 MB 1.2 MB/s eta 0:00:22
    --------------------------------------- 0.4/25.7 MB 1.2 MB/s eta 0:00:22
    --------------------------------------- 0.5/25.7 MB 1.2 MB/s eta 0:00:21
    --------------------------------------- 0.5/25.7 MB 1.2 MB/s eta 0:00:22
    -


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### rank_bm25 Installation

In [5]:
%pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### joblib Installation

In [6]:
%pip install joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### dill Installation

In [7]:
%pip install dill

   ---------------------------------------- 0.0/119.7 kB ? eta -:--:--
   --- ------------------------------------ 10.2/119.7 kB ? eta -:--:--
   ---------- ---------------------------- 30.7/119.7 kB 660.6 kB/s eta 0:00:01
   ------------- ------------------------- 41.0/119.7 kB 393.8 kB/s eta 0:00:01
   -------------------------- ------------ 81.9/119.7 kB 573.4 kB/s eta 0:00:01
   ------------------------------ -------- 92.2/119.7 kB 581.0 kB/s eta 0:00:01
   -------------------------------------- 119.7/119.7 kB 582.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Apply ir_datasets fix for encoding

In [8]:
%pip install scikit-learn

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     -------------------------------- ----- 51.2/60.8 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 647.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.7 MB 3.2 MB/s eta 0:00:04
    --------------------------------------- 0.1/10.7 MB 2.1 MB/s eta 0:00:06
    --------------------------------------- 0.3/10.7 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.4/10.7 MB 2.8 MB/s eta 0:00:04
   - -------------------------------------- 0.5/10.7 MB 2.9 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/10.7 MB 2.9 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/10.7 MB 3.0 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/10.7 MB 2.9 MB/s eta 0:00:04
   --- --


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os

# Replace the user's path with the correct path
ir_datasets_path = r"C:\Users\FSOS\AppData\Local\Programs\Python\Python311\Lib\site-packages\ir_datasets\formats\tsv.py"

In [11]:
# Check if file exists
if not os.path.exists(ir_datasets_path):
    print(f"Error: File not found at {ir_datasets_path}")
    exit()

# Read the file content
with open(ir_datasets_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Replace lines 25-28 with new code
    new_lines = [
        "            if isinstance(self.dlc, list):\n",
        "                self.stream = io.TextIOWrapper(self.ctxt.enter_context(self.dlc[self.stream_idx].stream()), encoding='utf-8')\n",
        "            else:\n",
        "                self.stream = io.TextIOWrapper(self.ctxt.enter_context(self.dlc.stream()), encoding='utf-8')\n"
    ]
    
    # Replace the lines (note: list indices are 0-based, so lines 25-28 are indices 24-27)
    lines[24:28] = new_lines
    
    # Write back the modified content
    with open(ir_datasets_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)
    print("Successfully updated the file encoding settings.")

Successfully updated the file encoding settings.


In [ ]:
pip install matplotlib